# Assignment 6

In [179]:
import numpy as np
import pandas as pd

## Data

In [196]:
Wall_Heigh = 3
Wall_Wide = 5
Area_Total = Wall_Heigh * Wall_Wide
Unit_Depth = 1
Unit_Heigh = 0.25
Unit_Area = Unit_Depth * Unit_Heigh
Temperature_Indoor = 20
Temperature_Outdoor = -10

Categories = ["Type","Area","h","k","L","RValue"]

Resistances_Wall_Names = ["R_Internal","R_Foam","R_Plaster1","R_Parallel","R_Plaster2","R_External"]
Resistances_Wall_Type = ["conv","cond","cond","cond","cond","conv"]
Resistances_Wall_Area = [Unit_Area,Unit_Area,Unit_Area,Unit_Area,Unit_Area,Unit_Area]
Resistances_Wall_h = [10,None,None,None,None,25]  
Resistances_Wall_k = [None,0.026,0.22,None,0.22,None]
Resistances_Wall_L = [None,0.03,0.02,0.16,0.02,None]
Resistances_Wall_RValue = [0,0,0,0,0,0]

resistance_ListOfLists_Wall = [Resistances_Wall_Type,Resistances_Wall_Area,Resistances_Wall_h,Resistances_Wall_k,Resistances_Wall_L,Resistances_Wall_RValue]
resistances_DF_Wall = pd.DataFrame(resistance_ListOfLists_Wall,index=Categories,columns=Resistances_Wall_Names)
resistances_DF_Wall = resistances_DF_Wall.transpose()

Resistance_Parallel_Names = ["R_Plaster_Brick1","R_Brick","R_Plaster_Brick2"]
Resistance_Parallel_Type = ["cond","cond","cond"]
Resistance_Parallel_Area = [Unit_Depth*0.015,Unit_Depth*0.22,Unit_Depth*0.015]
Resistance_Parallel_k = [0.22,0.72,0.22]
Resistance_Parallel_L = [0.16,0.16,0.16]
Resistance_Parallel_h = [None,None,None]
Resistance_Parallel_RValue = [0,0,0]

resistance_ListOfLists_Parallel = [Resistance_Parallel_Type,Resistance_Parallel_Area,Resistance_Parallel_h,Resistance_Parallel_k,Resistance_Parallel_L,Resistance_Parallel_RValue]
resistances_DF_Parallel = pd.DataFrame(resistance_ListOfLists_Parallel,index=Categories,columns=Resistance_Parallel_Names)
resistances_DF_Parallel = resistances_DF_Parallel.transpose()

In [201]:
resistances_DF_Wall

,Type,Area,h,k,L,RValue
R_Internal,conv,0.25,10,None,None,0
R_Foam,cond,0.25,None,0.026,0.03,0
R_Plaster1,cond,0.25,None,0.22,0.02,0
R_Parallel,cond,0.25,None,None,0.16,0
R_Plaster2,cond,0.25,None,0.22,0.02,0
R_External,conv,0.25,25,None,None,0


In [202]:
resistances_DF_Parallel

,Type,Area,h,k,L,RValue
R_Plaster_Brick1,cond,0.015,None,0.22,0.16,0
R_Brick,cond,0.22,None,0.72,0.16,0
R_Plaster_Brick2,cond,0.015,None,0.22,0.16,0


## Calculating the Resistance Value of the Parallel Resistances

In [269]:
IsCond = (resistances_DF_Wall.loc[:,"Type"] == "cond") & (resistances_DF_Wall.loc[:,"k"].notna())
IsConv = (resistances_DF_Wall.loc[:,"Type"] == "conv") & (resistances_DF_Wall.loc[:,"h"].notna())

In [270]:
def   RValue_cond (Serie):
    RValue = Serie["L"]/(Serie["Area"]*Serie["k"])
    return RValue

def   RValue_conv (Serie):
    RValue = 1./(Serie["Area"]*Serie["h"])
    return RValue

In [271]:
for i in resistances_DF_Wall[IsCond].index:
  resistances_DF_Wall.loc[i,"RValue"] = RValue_cond(resistances_DF_Wall.loc[i,:])

for i in resistances_DF_Wall[IsConv].index:
  resistances_DF_Wall.loc[i,"RValue"] = RValue_conv(resistances_DF_Wall.loc[i,:])

In [273]:
#We can do this defining a function and using a for or with the next definitions:
#resistances_DF_Wall.loc[IsCond,"RValue"] = resistances_DF_Wall[IsCond]["L"]/(resistances_DF_Wall[IsCond]["Area"] * resistances_DF_Wall[IsCond]["k"])
#resistances_DF_Wall.loc[IsConv,"RValue"] = 1./(resistances_DF_Wall[IsConv]["Area"] * resistances_DF_Wall[IsConv]["h"])

for i in resistances_DF_Parallel.index:
  if resistances_DF_Parallel.loc[i,"Type"] == "cond":
    RValue_i = RValue_cond(resistances_DF_Parallel.loc[i,:])
    resistances_DF_Parallel.loc[i,"RValue"] = RValue_i

resistances_DF_Wall.loc["R_Parallel","RValue"] = 1./(np.power(resistances_DF_Parallel.loc[:,"RValue"],-1).sum())

In [274]:
resistances_DF_Parallel

,Type,Area,h,k,L,RValue
R_Plaster_Brick1,cond,0.015,None,0.22,0.16,48.4848
R_Brick,cond,0.22,None,0.72,0.16,1.0101
R_Plaster_Brick2,cond,0.015,None,0.22,0.16,48.4848


In [275]:
resistances_DF_Wall

,Type,Area,h,k,L,RValue
R_Internal,conv,0.25,10,None,None,0.4
R_Foam,cond,0.25,None,0.026,0.03,4.61538
R_Plaster1,cond,0.25,None,0.22,0.02,0.363636
R_Parallel,cond,0.25,None,None,0.16,0.969697
R_Plaster2,cond,0.25,None,0.22,0.02,0.363636
R_External,conv,0.25,25,None,None,0.16


In [292]:
R_Total_Wall = resistances_DF_Wall["RValue"].sum()
#R_Total_Wall = np.sum(resistances_DF_Wall["RValue"])
R_Total_Wall

6.872354312354313

In [293]:
Q_Unit = (Temperature_Indoor - Temperature_Outdoor)/R_Total_Wall
Q_Total = Q_Unit * Wall_Heigh/Unit_Heigh * Wall_Wide

In [290]:
print("The Rate of Heat Transfer through the wall is "+str(round(Q_Total,2))+" Watts")

The Rate of Heat Transfer through the wall is 261.92 Watts
